# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [2]:
df=pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df=df[['id','group','action']]

In [4]:
df.groupby('id').sum().value_counts()

group                 action   
control               view         2400
experiment            view         2068
controlcontrol        viewclick     932
experimentexperiment  viewclick     928
dtype: int64

In [5]:
"""
This shows that there is no overlap between control and experimental groups. If there were, we could see some
group values listed as controlexperiment or experimentcontrol.
We also know that there is no one who clicked and did not view, since if this was the case we would see value counts for
the action click, in the above grouping, whereas we only see 'view' and 'viewclick' listed.

Inspection of the data shows that the experimental group was significantly
smaller than the control group but the conversion rate was only slightly smaller.
This suggests that the control experimental group might have a significantly
higher conversion rate. However, we need to run a chi-square test to see
if we get a p-value below a 0.05 threshhold.
"""

"\nThis shows that there is no overlap between control and experimental groups. If there were, we could see some\ngroup values listed as controlexperiment or experimentcontrol.\nWe also know that there is no one who clicked and did not view, since if this was the case we would see value counts for\nthe action click, in the above grouping, whereas we only see 'view' and 'viewclick' listed.\n\nInspection of the data shows that the experimental group was significantly\nsmaller than the control group but the conversion rate was only slightly smaller.\nThis suggests that the control experimental group might have a significantly\nhigher conversion rate. However, we need to run a chi-square test to see\nif we get a p-value below a 0.05 threshhold.\n"

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
#Your code here
obs=np.array([[2400, 932],[2068, 928]])

result = stats.contingency.chi2_contingency(obs)
print(result)

(6.712921132285345, 0.009571680497042266, 1, array([[2352.61946903,  979.38053097],
       [2115.38053097,  880.61946903]]))


In [7]:
expected_conversion_rate=result[3][1][1]/(result[3][1][0]+result[3][1][1])
expected_conversion_rate

0.29393173198482936

In [8]:
conversion_rate_experimental = 928/(2068+928)
print('expected conversion rate: ', expected_conversion_rate)
print('experimental group conversion rate: ', conversion_rate_experimental)
print('difference in conversion rates:', conversion_rate_experimental-expected_conversion_rate)

expected conversion rate:  0.29393173198482936
experimental group conversion rate:  0.3097463284379172
difference in conversion rates: 0.01581459645308786


In [9]:
"""
Since our p value is around 0.001, and our alpha was set at 0.05 we should
see this result as significant with the conversion rate for the experimental
group at 1.58% above the overall group average.
"""

'\nSince our p value is around 0.001, and our alpha was set at 0.05 we should\nsee this result as significant with the conversion rate for the experimental\ngroup at 1.58% above the overall group average.\n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [10]:
grouped=df.groupby('id').sum()

In [11]:
grouped.head()

,group,action
id,,
182988,experiment,view
182994,controlcontrol,viewclick
183089,control,view
183136,experiment,view
183141,experimentexperiment,viewclick


In [12]:
grouped.value_counts()

group                 action   
control               view         2400
experiment            view         2068
controlcontrol        viewclick     932
experimentexperiment  viewclick     928
dtype: int64

In [13]:
control_conversion = grouped[(grouped['group'].str.contains('control'))&(grouped['action']=='viewclick')].value_counts().tolist()[0]
control_conversion

932

In [14]:
control_total = np.array(grouped[grouped['group'].str.contains('control')].value_counts().tolist()).sum()
control_total

3332

In [15]:
control_conversion_rate = control_conversion/control_total
control_conversion_rate

0.2797118847539016

In [16]:
total_experimental_group = np.array(grouped[grouped['group'].str.contains('experiment')].value_counts().tolist()).sum()
experiment_expected_conversion = total_experimental_group*control_conversion_rate

In [17]:
experiment_expected_conversion

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [18]:
#Your code here
std = np.sqrt(control_total*control_conversion_rate*(1-control_conversion_rate))
std

25.909622216646923

In [20]:
z_score = (928-experiment_expected_conversion)/std

In [21]:
z_score

3.4729643112857396

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [22]:
#Your code here
stats.norm.sf(z_score)

0.00025737189895806245

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **This p value is smaller than the p value of the previous statistical test, though both tests show significance well below an alpha of 0.05. This is because the previous statistical test was a contingency test that compared two groups of sample data, whereas the chi square test above treats the control group as a population that the experimental group is being compared against. I think that the chi square contingency test is preferable, since  our control data is a sample, rather than a population, and therefore we want a higher p value.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.